In [ ]:
# %%
# Install if needed (uncomment if required)
# !pip install pinecone sentence-transformers python-dotenv pandas tqdm

In [ ]:
# %%
import os
import pandas as pd
from dotenv import load_dotenv
from pinecone import Pinecone
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from ollama import Client

In [2]:
# %%
# Load environment variables
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [3]:
# %%
# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("funding-search-bge")

In [4]:
# %%
# Sample user query
query = "We are an AI company focused on AI for robotics. We are focusing on research right now."

In [5]:
# %%
# Embed query using local BGE model
model = SentenceTransformer("BAAI/bge-small-en")
query_embedding = model.encode(query).tolist()

In [6]:
# %%
# Query Pinecone using the embedded query vector
semantic_matches = index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True,
    namespace="open-source-v1"
)

In [7]:
# %%
# Format matches into readable funding blocks
def generate_structured_funding_blocks(matches, user_query: str) -> str:
    formatted_blocks = []

    field_aliases = {
        "Amount": ["amount", "how much", "funding", "money"],
        "Deadline": ["deadline", "last date", "until", "submission date"],
        "Eligibility": ["eligible", "eligibility", "who can apply"],
        "Procedure": ["procedure", "how to apply", "application", "steps", "process"],
        "Contact": ["contact", "email", "person", "support"],
    }

    for idx, match in enumerate(matches, start=1):
        meta = match["metadata"]
        name = meta.get("name", "Unnamed")

        fields = {
            "Description": meta.get("description"),
            "Domain": meta.get("domain"),
            "Eligibility": meta.get("eligibility"),
            "Amount": meta.get("amount"),
            "Deadline": meta.get("deadline"),
            "Procedure": meta.get("procedure"),
            "Contact": meta.get("contact"),
            "URL": meta.get("url"),
        }

        missing_fields = []
        for key, value in fields.items():
            if key in field_aliases:
                if any(alias in user_query.lower() for alias in field_aliases[key]):
                    if not value or "not found" in str(value).lower():
                        missing_fields.append(key)

        block = f"""**{idx}. {name}**\n"""
        for key in ["Description", "Domain", "Eligibility", "Amount", "Deadline", "Procedure", "Contact"]:
            val = fields[key]
            if val and "not found" not in str(val).lower():
                block += f"   - **{key}**: {val}\n"

        if missing_fields:
            block += f"   - *Couldn't trace information about {', '.join(missing_fields)}.*\n"

        if fields["URL"]:
            block += f"   - **For more information visit**: {fields['URL']}\n"

        formatted_blocks.append(block)

    return "\n".join(formatted_blocks)

In [8]:
# %%
semantic_output = generate_structured_funding_blocks(semantic_matches["matches"], query)
print(semantic_output)

**1. MASTER 2nd Open Call**
   - **Description**: MASTER supports projects that develop and validate innovative XR-based educational content for robotics training. The aim is to create practical training scenarios that promote the use of XR technologies in industrial robotics and give students and specialists access to modern learning tools.  Further information can be found here
   - **Domain**: VR/XR
   - **Eligibility**: Universities, research institutions, educational institutions, SMEs and large companies
   - **Amount**: Up to 100,000 euros
   - **Deadline**: June 12, 2025
   - **For more information visit**: https://www.master-xr.eu/open-calls/open-call-2/

**2. Promotion of interdisciplinary pilot projects on the topic of "Neurobiologically inspired artificial intelligence"**
   - **Description**: Guideline for the funding of interdisciplinary pilot projects on the topic of "Neurobiologically inspired artificial intelligence" dated: 25.11.2024 Federal Ministry of Education and 

In [ ]:
# # %%
# # Prepare the LLM prompt
# llm_prompt = f"""
# The company described itself as:

# "{query}"

# Here are the top 5 most relevant public funding programs in Germany, based on a semantic search match to their needs:

# {semantic_output}

# Now:

# Please write a concise and professional recommendation containing **only the top 2–3 most relevant funding programs** in this format:

# Only select the top programs that most directly match the company’s domain, maturity stage (e.g., early-stage research), or funding needs. Ignore entries that are vague or poorly aligned.

# For each recommendation, follow this format exactly:

# 1. <Program Name>  
# Why it fits: <1–2 lines explaining relevance to the company’s domain (or) industry (or) field of work>  
# **Description**: <1–3 sentence summary of what the program funds and its focus>  
# **Domain**: <Domain>  
# **Eligibility**: <Eligibility>  
# **Amount**: <Amount>  
# **Deadline**: <Deadline>  
# **Contact**: <Contact person, email, or organization>  
# **Next Steps**:  
# - <Step 1> Visit the official call page: <Program URL>  
# - <Step 2> Provide one or two additional helpful steps based on available info (e.g., forming a consortium, preparing documents, contacting support, etc.)  
# - <Step 3> (Optional) Include any additional steps if mentioned in the procedure  

# If any field like **Amount**, **Deadline**, **Eligibility**, **Procedure**, **Contact**, or **Domain** is missing, either omit the line or say “Not specified”.

# Use simple bullet points under **Next Steps**. Only list the top 2 or 3 programs — not all 5.
# """

In [16]:
# # Save the prompt
# with open("llm_prompt_llama.txt", "w") as f:
#     f.write(llm_prompt)

In [17]:
# # Load the prompt
# with open("llm_prompt.txt", "r") as f:
#     llm_prompt = f.read()

In [ ]:
# Prepare the LLM prompt
llm_prompt = f"""
The company described itself as:

"{query}"

Here are the top 5 most relevant public funding programs in Germany, based on a semantic search match to their needs:

{semantic_output}

Now:

Please write a concise and professional recommendation containing **only the top 2–3 most relevant funding programs** in this format:

Only select the top programs that most directly match the company’s domain, maturity stage (e.g., early-stage research), or funding needs. Ignore entries that are vague or poorly aligned.

For each recommendation, follow this format exactly:

1. <Program Name>  
Why it fits: <1–2 lines explaining relevance to the company’s domain (or) industry (or) field of work>  
**Description**: <1–3 sentence summary of what the program funds and its focus>  
**Domain**: <Domain>  
**Eligibility**: <Eligibility>  
**Amount**: <Amount>  
**Deadline**: <Deadline>  
**Contact**: <Contact person, email, or organization — not the URL>  
**Next Steps**:  
- Step 1: <Visit the official call page>  
- Step 2: <Provide one or two helpful next steps, e.g., form a consortium, prepare documents>  
- Step 3: (Optional) <Any extra steps from the program's procedure>

If any field like **Amount**, **Deadline**, **Eligibility**, **Procedure**, **Contact**, or **Domain** is missing, either omit the line or say “Not specified”.

Make sure **Next Steps** are formatted as real bullet points on new lines — not in a single line. Use simple and readable language. List only the top 2 or 3 matching programs — not all 5.
"""

In [23]:
# %%
# Run the prompt using LLaMA 3.2 via Ollama
client = Client(host="http://localhost:11434")

response = client.generate(
    model="llama3.2",
    prompt=llm_prompt,
    stream=False
)

In [24]:
# %%
# Output the final result
print("\n🧾 LLaMA 3.2 Recommendation:\n")
print(response['response'])


🧾 LLaMA 3.2 Recommendation:

Here is a concise and professional recommendation containing the top 2-3 most relevant funding programs for the AI company focused on AI for robotics:

1. **FORTIS 1st Open Call**

Why it fits: This program supports projects that develop innovative solutions for safe, trustworthy, and efficient human-robot interaction in industrial applications, aligning with the company's domain of AI for robotics.

**Description**: FORTIS funds projects that integrate multimodal communication methods and digital twins to optimize collaboration between humans and robots in areas such as construction, maintenance, and logistics. The aim is to promote practical technologies that improve interaction and increase productivity.

**Domain**: Human-robot interaction (HRI), multimodal communication, digital twins

**Eligibility**: Consortia of 2-3 organizations: Start-ups, SMEs, mid-caps, research institutions, universities

**Amount**: Up to 250,000 euros

**Deadline**: June 04,

In [18]:
# %%
# Run the prompt using LLaMA 3.2 via Ollama
client = Client(host="http://localhost:11434")

response = client.generate(
    model="llama3.2",
    prompt=llm_prompt,
    stream=False
)

In [19]:
# %%
# Output the final result
print("\n🧾 LLaMA 3.2 Recommendation:\n")
print(response['response'])



🧾 LLaMA 3.2 Recommendation:

Based on the provided information, I recommend the following top funding programs for the AI company focused on AI for robotics:

1. FORTIS 1st Open Call
Why it fits: This program supports projects that develop innovative solutions for safe, trustworthy, and efficient human-robot interaction in industrial applications.
**Description**: The aim is to promote practical technologies that improve interaction and increase productivity through the integration of multimodal communication methods and digital twins.
**Domain**: Human-robot interaction (HRI), multimodal communication, digital twins
**Eligibility**: Consortia of 2-3 organizations: Start-ups, SMEs, mid-caps, research institutions, universities
**Amount**: Up to 250,000 euros
**Deadline**: June 04, 2025
**Contact**: For more information visit: https://fortis-project.eu/open-call-1/
Next Steps:
• Visit the official call page: https://fortis-project.eu/open-call-1/
• Review and prepare consortium documen